In [38]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import sqlite3
import datetime
from datetime import date
import pandas as pd
from collections import Counter 

In [2]:
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
%matplotlib inline

## Mobile, Alabama News Sources

In [3]:
def al_data_title_scraper():
    al_data = urlopen('https://www.al.com/')
    al_soup = BeautifulSoup(al_data, 'html.parser')
    articles = al_soup.find_all('article')
    regex_pattern = r'\d+[hm]\sago'
    titles = []
    
    for i in articles:
        titles.append(i.text)
        
    titles = [x.strip(regex_pattern).lstrip().rstrip() for x in titles]
    titles = [i for i in set(titles)]
    
    return titles

In [4]:
def fox_10_news_scraper():
    fox_10_news = urlopen('https://www.fox10tv.com/')
    fox_10_news_soup = BeautifulSoup(fox_10_news, 'html.parser')
    fox_titles = fox_10_news_soup.find_all('a', class_='tnt-asset-link')
    remove_list = ['Tracking the Tropics', '1st & 10','COVID-19 Headlines', 'Back to School', 'News Now Update', 'Our Apps', 'News Now Livestream']
    regex_pattern = r'\d:\d\d'
    
    local_fox_titles = []
    for i in fox_titles:
        local_fox_titles.append(i.text.rstrip().lstrip())
        
    final_fox_titles = []
    for i in local_fox_titles:
        if i == '' or re.match(regex_pattern, i):
            pass
        elif i in remove_list:
            pass
        else:
            final_fox_titles.append(i)
            
    return final_fox_titles

## Birmingham, Alabama News Scrapers

In [5]:
### Birmingham, Alabama
def wbrc_scraper():
    wbrc_birmingham = urlopen('https://www.wbrc.com/')
    wbrc_soup = BeautifulSoup(wbrc_birmingham, 'html.parser')
    new_titles = wbrc_soup.find_all('a',class_='unstyled-link')
    
    cleaned_titles = []
    
    for i in new_titles:
        cleaned_titles.append(i.text.rstrip().lstrip())
        
    return cleaned_titles

In [6]:
def al_birmingham_scraper():
    al_birmingham = urlopen('https://www.al.com/birmingham/')
    al_soup = BeautifulSoup(al_birmingham, 'html.parser')
    articles = al_soup.find_all('article')

    for i in articles:
        regex_pattern = r'\d+[hm]\sago'
        titles = []
    
        for i in articles:
            titles.append(i.text)
        
        titles = [x.strip(regex_pattern).lstrip().rstrip() for x in titles]
        titles = [i for i in set(titles)]
    
        return titles

In [46]:
al_birmingham_scraper()

["What Nick Saban had to say following Alabama's 41-24 win over Georgia and former Asst. Kirby Smart",
 'How to get a good night’s sleep: Dear Annie',
 "Which Hard Seltzer Is The Best? | Bless Your RankMatt tackles the huge new craze of spiked seltzers. Watch him try as many as he can stand so you don't have to!",
 'Woman indicted in crash that killed elderly Tuscaloosa couple48m',
 'Over 1,300 cast ballots in ‘historic’ Jeffco Saturday voting1h',
 'A few extra observations, context to Alabama’s win over GeorgiaAlabama beat No. 3 Georgia 41-24 on Saturday night. We take a second look at things every Sunday.23h',
 'Ratcliffe: NY Post’s Hunter Biden story isn’t Russian disinformation2h',
 'Auto supplier expanding in two Alabama cities, creating 182 jobsInternational Automotive Components is based in Luxembourg.6h',
 'Amber Alert cancelled; 2 Mississippi boys found safePolice officials say Kaiden Wall, 6, and Kolden Wall, 2, are safe and their mother, 33-year-old Sarah Lynn Caswell, is in

### Huntsville, Alabama - News Scrapers

In [ ]:
### Grab news from Huntsville


### Database Functions (Will be moved)

In [7]:
def show_all_data():
    con = sqlite3.connect('news.db')
    cursor = con.cursor()
    cursor.execute('SELECT * from news_titles')
    titles = cursor.fetchall()
    con.close()
    return titles

In [8]:
def grab_titles():
    title_list = []
    dirty_data = pd.Series(show_headlines())
    for i in dirty_data:
        title_list.append(i[0])
    return pd.Series(title_list)

In [9]:
def show_headlines():
    con = sqlite3.connect('news.db')
    cursor = con.cursor()
    cursor.execute('SELECT article_title from news_titles')
    titles = cursor.fetchall()
    con.close()
    return titles

In [10]:
def add_headlines(titles, source):
    today = date.today()
    weekday = datetime.datetime.now().strftime("%A")
    for i in titles:
        con = sqlite3.connect('news.db')
        cursor = con.cursor()
        cursor.execute('INSERT INTO news_titles (article_title, source, weekday, date) VALUES (?,?,?,?)', (i, source, weekday, today))
        con.commit()
        
    con.close()

In [11]:
def run_all():
    al_headlines_main = al_data_title_scraper()
    add_headlines(al_headlines_main, 'www.al.com')
    fox_10_headlines = fox_10_news_scraper()
    add_headlines(fox_10_headlines, 'www.fox10tv.com')
    wbrc_headlines = wbrc_scraper()
    add_headlines(wbrc_headlines, 'www.wbrc.com/')
    
    return 'Program has sucessfully scraped and saved the data.'

In [12]:
run_all()

'Program has sucessfully scraped and saved the data.'

### Observing the Data that we have collected

In [117]:
all_titles = grab_titles()

In [14]:
new_str = ''
for i in all_titles:
    new_str = new_str + i + '\n'

In [118]:
meta_alabama = 0
alabama_titles = []
football_count = 0
football_titles = []
covid_count = 0
covid_titles = []
trump_count = 0
trump_titles = []
saban_count = 0
saban_titles = []
misc_titles = []
for i in set(all_titles):
    if 'Alabama' in i:
        meta_alabama +=1
        alabama_titles.append(i)
    elif 'football' in i:
        football_count += 1
        football_titles.append(i)
    elif 'Trump' in i:
        trump_count += 1
        trump_titles.append(i)
    elif 'COVID' in i:
        covid_count += 1
        covid_titles.append(i)
    elif 'Nick Saban' or 'Saban' in i:
        saban_count += 1
        saban_titles.append(i)
    else:
        misc_titles.append(i)

In [119]:
len(set(all_titles))

1738

In [120]:
meta_alabama

275

In [121]:
football_count

47

In [122]:
covid_count

82

In [123]:
trump_count

75

In [124]:
saban_count

1259

In [125]:
misc_titles

[]

In [61]:
### There will be a few list that will make up the entire sports spectrum
### For example, there will be football words, baseball words, basketball words, etc.

# football_positions = ['QB','RB','FB','WR','OT','C','G','TE','DE','DT', 'LB', 'OLB', 'S', 'CB', 'FS']

football_words = ['Saban', 'ESPN', 'Alabama football', 'Tide', 'Tide', 'football', 'NFL', 'Auburn football',
                 'Nick Saban', 'cornerback', 'vs.', 'players', 'games', 'College Football']

crime_words = ['murder', 'shooting', 'killed', 'dead', 'injured', 'homicide', 'gunfire', 'injured', 'beaten', 'death',
              'arrested', 'abuse', 'charged']